In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import skew, kurtosis, mode
from scipy.spatial import KDTree
from scipy.ndimage import convolve
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

global vectors
vectors = []

In [ ]:

def trim_cloud(data,start,stop):
    floor = mode(data[:, 2])[0] - 10
    data[:,2] = floor - data[:, 2] 
    data = data[data[:, 2] > 0]
    data = data[
    (data[:,0] >= stop) & 
    (data[:,0] <= start) 
    ]
    return data

def core_props(tree,data,x_span = 1, y_span = 12,y_index = 0, step = 1):
    x_values = np.unique(data[:, 0])
    core_types = {}
    y = y_index
    x = x_values.min()
    while(x < x_values.max()):
        distribution = get_distribution(tree,data,[x, y], x_span, y_span) 
        core_types[x] = get_props(distribution)
        vectors.append(core_types[x])
        x += step
    return core_types

def get_distribution(tree,cloud,point=[0, 0], x_span=10, y_span=15):
    search_radius = max(x_span, y_span)
    
    indices = tree.query_ball_point(point, search_radius)
    
    result_points = cloud[indices]

    filtered_points = result_points[
        (result_points[:, 0] >= point[0] - x_span) & (result_points[:, 0] <= point[0] + x_span) &
        (result_points[:, 1] >= point[1] - y_span) & (result_points[:, 1] <= point[1] + y_span)
    ]
    
    return filtered_points[:, 2]

def get_props(distribution):
    properties = []

    mean = np.mean(distribution)
    variance =  np.var(distribution)
    std = np.std(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    properties.append(mean)
    properties.append(variance)
    properties.append(std)
    properties.append(skw)
    properties.append(kurt)
    
    return properties

def plot(plots,tree,data,y_index, x_span, y_span, step):

    properties = core_props(tree,data,x_span, y_span, y_index, step)
    index = 0
    for i in range(5):
        for j in range(i + 1):
            plots[index].scatter([prop[j] for prop in properties.values()], [prop[i] for prop in properties.values()])
            index += 1



In [ ]:
def scatter_plot(file_path,plots,x_span,y_span,step):
    coords_file = os.path.join(file_path, '.component_parameters.txt')
    with open(coords_file) as file:
            lines = file.readlines()
            for line in lines:
                if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                    y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                    x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                    x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())
    
    point_cloud = []

    if os.path.isdir(file_path):
        lidar_files = [fn for fn in os.listdir(
            file_path) if fn.endswith('.bpc')]
        if any(lidar_files):
            lidar_filename = file_path + os.sep + lidar_files[0]

    data = np.fromfile(lidar_filename, dtype=np.float32)
    point_cloud = data.reshape(-1, 3)  # to xyz

    ff = ~np.isnan(point_cloud).any(axis=1)
    point_cloud = point_cloud[ff, ...]

    point_cloud[:, 1] = point_cloud[:, 1] - float(y_offset)

    print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")

    matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
    with open(matrix_file, 'r') as file:
        lines = file.readlines()

    transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

    num_points = point_cloud.shape[0]

    homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
    transformed_points = homogeneous_points @ transformation_matrix.T
    point_cloud = transformed_points[:, :3]
    point_cloud = trim_cloud(point_cloud,x_start,x_stop)
    point_tree = KDTree(point_cloud[:,:2])

    plot(plots,point_tree,point_cloud,y_offset,x_span,y_span,step)

def find_parts(directory_path):
    parts_paths = []

    # Walk through the directory structure
    for root, dirs, files in os.walk(directory_path):
        for folder in dirs:
            folder = os.path.join(root,folder)
            if ("Box1\AdaptiveZ_10mm" in folder) and folder.endswith("_4"):
                parts_paths.append(folder)

    return parts_paths

In [ ]:

base_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF\CH\Macassa_clearance"

paths = find_parts(base_path)

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(5, 5, width_ratios=[1,1,1,1,1], height_ratios=[1, 1, 1, 1, 1], wspace=0.3, hspace=0.3)
plots = []
names = ["Mean", "Variance", "Standard Deviation", "Skewness", "Kurtosis"]
index = 0

for i in range(5):
        for j in range(i + 1):
            plots.append(fig.add_subplot(gs[i, j]))
            plots[index].set_ylabel(f'{names[i]}')
            plots[index].set_xlabel(f'{names[j]}')
            index += 1

for path in paths:
    scatter_plot(path,plots,1,15,1)    



plt.show()

